In [18]:
import torch as torch
import torch.optim as optim
import torch.nn as nn
import os
import torchvision
import torch.nn as nn
from torch.autograd import Variable as var
import logging as log
import gc
import numpy as np
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset
import torch.multiprocessing as mp
import torch.backends.cudnn as cudnn
from PIL import Image
from glob import glob
import sys
import os
import time
import torchvision.datasets as datasets

In [21]:
PARAMETERS = {}
PARAMETERS['model_name'] = 'resnet18'
PARAMETERS['model_saved'] = "/kaggle/working/"
PARAMETERS['learning_rate'] = 0.05
PARAMETERS['momentum'] = 0.9
PARAMETERS['epochs'] = 100
PARAMETERS['weight_decay'] = 0.0001
PARAMETERS['batch_size'] = 128
PARAMETERS['temperature'] = 0.07
PARAMETERS['num_channels'] = 3
PARAMETERS['dictionary_size'] = 8192
PARAMETERS['num_workers'] = 1
PARAMETERS['num_cores'] = 1
PARAMETERS['log_steps'] = 20
PARAMETERS['load_from_saved'] = False
PARAMETERS['start_epoch'] = 1

In [22]:
class BaseModel(nn.Module):
    def __init__(self, base_model_name, channels_out):
        super(BaseModel, self).__init__()

        if base_model_name == 'resnet50':
          model = models.resnet50(pretrained=False)
        elif base_model_name == 'resnet18':
          model = models.resnet18(pretrained=False)
        
        penultimate = model.fc.weight.shape[1]
        modules = list(model.children())[:-1]
        self.encoder = nn.Sequential(*modules)

        self.relu = nn.ReLU()
        self.fc = nn.Linear(penultimate, channels_out);
    
    def forward(self,x):
        x = self.encoder(x)
        x = x.view(x.size(0),-1)
        x = self.relu(x)
        x = self.fc(x)
        
        return x

In [27]:
class PneumoniaClassifierModel(nn.Module):
    def __init__(self, encoder):
        super().__init__() 
        self.encoder = encoder 
        for params in self.encoder.parameters() :
            params.requires_grad = False 

        encoder_sortie = self.encoder.fc.weight.shape[0]
        self.relu1 = nn.ReLU()
        self.fc = nn.Linear(encoder_sortie, 512)
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(512, 512)
        self.relu3 = nn.ReLU()
        self.fc3 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.encoder(x)
        x = self.relu1(x)
        x = self.fc(x)
        x = self.relu2(x)
        x = self.fc2(x)
        x = self.relu3(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [23]:
pth = "/kaggle/input/moco-v2/trained_encoder.pth"
encoder_model = BaseModel('resnet18', 128)
encoder_model.load_state_dict(torch.load(pth)["model_state_dict"])

<All keys matched successfully>

In [25]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
augmentation = [
            transforms.RandomResizedCrop(224, scale=(0.2, 1.)),
            transforms.ToTensor(),
            normalize
               ]
#Training dataset
class_train_dataset = datasets.ImageFolder(
        "/kaggle/input/chest-xray-pneumonia/chest_xray/train/",
         transforms.Compose(augmentation)
                                          )

#Training dataloader
class_train_loader = torch.utils.data.DataLoader(
    class_train_dataset,
    batch_size = PARAMETERS['batch_size'],
    shuffle=True
                                                )
#Validation dataset
class_val_dataset = datasets.ImageFolder(
        "/kaggle/input/chest-xray-pneumonia/chest_xray/val/",
         transforms.Compose(augmentation)

                                        )
#Validation dataloader
class_val_loader = torch.utils.data.DataLoader(
    class_val_dataset,
    shuffle=True
                                               )

class_test_dataset = datasets.ImageFolder(
        "/kaggle/input/chest-xray-pneumonia/chest_xray/test/",
         transforms.Compose(augmentation)

                                        )
#Validation dataloader
class_test_loader = torch.utils.data.DataLoader(
    class_test_dataset,
    shuffle=True
                                               )



In [30]:
torch.cuda.empty_cache()
classification_model = PneumoniaClassifierModel(encoder_model)
classification_model = classification_model.cuda(0)

loss_function = nn.BCELoss().cuda()
optimizer = torch.optim.Adam(classification_model.parameters())

In [34]:
num_epochs = 30
start = 30 
for epoch in range(num_epochs): 
    running_loss = 0.0
    running_acc = 0.0
    
    epoch_acc  = 0.0
    epoch_loss = 0.0
    
    nb_elts = 0
    classification_model.train()
    for i, bc in enumerate(class_train_loader):
        
        inputs, labels = bc[0] , bc[1] 
        #Load images and labels to GPU
        inputs = inputs.cuda(0)
        labels = labels.cuda(0)
            
        optimizer.zero_grad()
        # Forward pass
        outputs = classification_model(inputs)
        loss = loss_function(outputs.flatten(), labels.float())
        
        # Backward pass and optimization step
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_loss += loss.item()
        
        predictions = (outputs > 0.5).long().flatten()
        
        correct = (predictions == labels).sum().item()
        
        epoch_acc += correct
        running_acc += correct/predictions.size(0)
        
        
        nb_elts += predictions.size(0)
        if((i+1) % 10 == 0):
            print('Batch =({}) Acc =({})  Loss={:.5f} Time={}'.format(
               i, running_acc/10, running_loss/10, time.asctime()), flush=True)
            running_loss = 0.0
            running_acc = 0.0
            
            
        
     
    epoch_acc = epoch_acc / nb_elts
    print("Epoch: "+str(start + epoch)+ " Acc: "+str(epoch_acc)+ "Loss: "+str(epoch_loss))
    #Passage en mode évaluation pour le validation set
    classification_model.eval()
    val_acc = 0
    val_loss = 0
    for inputs, label in class_test_loader :
        inputs = inputs.cuda(0)
        label = label.cuda(0)        

        target = classification_model(inputs).flatten()
        loss = loss_function(target,label.float())
        prediction = (target > 0.5).long().flatten()
        val_acc += (prediction == label).item() * 1
        val_loss += loss.item()
    print('Epoch: {} Val Acc ={:.2f}  Val Loss={:.5f}'.format(
                   epoch + start, val_acc/len(class_test_loader), val_loss, flush=True))
    print("\n")  

Batch =(9) Acc =(0.97734375)  Loss=0.07139 Time=Thu Dec 29 19:31:06 2022
Batch =(19) Acc =(0.97265625)  Loss=0.06793 Time=Thu Dec 29 19:31:30 2022
Batch =(29) Acc =(0.965625)  Loss=0.08409 Time=Thu Dec 29 19:31:55 2022
Batch =(39) Acc =(0.95703125)  Loss=0.09983 Time=Thu Dec 29 19:32:20 2022
Epoch: 30 Acc: 0.9681748466257669Loss: 3.33422389999032
Epoch: 30 Val Acc =0.87  Val Loss=658.19786


Batch =(9) Acc =(0.9625)  Loss=0.08882 Time=Thu Dec 29 19:32:59 2022
Batch =(19) Acc =(0.96875)  Loss=0.08399 Time=Thu Dec 29 19:33:23 2022
Batch =(29) Acc =(0.96484375)  Loss=0.08696 Time=Thu Dec 29 19:33:47 2022
Batch =(39) Acc =(0.96015625)  Loss=0.09229 Time=Thu Dec 29 19:34:10 2022
Epoch: 31 Acc: 0.964148773006135Loss: 3.588732611387968
Epoch: 31 Val Acc =0.86  Val Loss=1119.96233


Batch =(9) Acc =(0.965625)  Loss=0.08430 Time=Thu Dec 29 19:34:48 2022
Batch =(19) Acc =(0.95859375)  Loss=0.09389 Time=Thu Dec 29 19:35:12 2022
Batch =(29) Acc =(0.9703125)  Loss=0.07234 Time=Thu Dec 29 19:35:35 2

In [33]:
torch.save({
              'epochs' : 30,
              'model_state_dict': classification_model.state_dict(),
              'optimizer_state_dict' : optimizer.state_dict()
            }, "/kaggle/working/classification_model_train_e30.pth")